<a href="https://colab.research.google.com/github/SeunghyeChae/tmdhey/blob/main/for_medical_get%26save_ect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mount/ install/ import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydicom
!pip install SimpleITK

     |████████████████████████████████| 2.0 MB 16.0 MB/s 
     |████████████████████████████████| 48.4 MB 2.0 MB/s 


In [33]:
import numpy as np
import SimpleITK as sitk
import pydicom
import matplotlib.pyplot as plt


# get array / save dcm
### (nii 3d 저장 / dcm 슬라이스 생성): itk snap에서 영상 뜨는 것 확인했음

In [ ]:
org= mhd경로

In [25]:
reader=sitk.ImageFileReader()
reader.SetFileName(org)
img=reader.Execute()
array=sitk.GetArrayFromImage(img)
# array=sitk.GetArrayViewFromImage(img)
# array

In [74]:
plt.imshow(array[120,:,:])

In [47]:
# 이건 바로 됨..! nii 3d 저장
Image = sitk.ReadImage(org)
sitk.WriteImage(Image, 'd.nii')

In [ ]:
imgClass = sitk.GetImageFromArray(array)
sitk.WriteImage(imgClass, 'd.nii.gz')

In [ ]:
def crop4(path, filename):
    file_name, d = os.path.splitext(filename)
    img = sitk.ReadImage(os.path.join(path, filename))
    imgarray = sitk.GetArrayFromImage(img)[0]
    w = int(imgarray.shape[0]/2)
    h = int(imgarray.shape[1]/2)
#左上
    new1 = imgarray[0:w, 0:h]
    new1 = new1[np.newaxis, :]
    out1 = sitk.GetImageFromArray(new1)
    result1 = os.path.join(path, file_name+'_1.dcm')
    sitk.WriteImage(out1, result1)

In [53]:
# mhd to dcm

img = sitk.ReadImage(org)
imgarray = sitk.GetArrayFromImage(img)[0]
w = int(imgarray.shape[0])
h = int(imgarray.shape[1])

new1 = imgarray[0:w, 0:h]
new1 = new1[np.newaxis, :]
out1 = sitk.GetImageFromArray(new1)
result1 = '_1.dcm'
sitk.WriteImage(out1, result1)

In [64]:
# imgarray가 array에 저장된 값이고
# img array로 바꾸면 121개의 슬라이스가 생김
# for imgarray ~~~ [0부터 120까지 해야댐]

imgarray.shape

(512, 512)

In [71]:
sitk.GetArrayFromImage(img)[120]

array([[-3024, -3024, -3024, ..., -3024, -3024, -3024],
       [-3024, -3024, -3024, ..., -3024, -3024, -3024],
       [-3024, -3024, -3024, ..., -3024, -3024, -3024],
       ...,
       [-3024, -3024, -3024, ..., -3024, -3024, -3024],
       [-3024, -3024, -3024, ..., -3024, -3024, -3024],
       [-3024, -3024, -3024, ..., -3024, -3024, -3024]], dtype=int16)

# ----- 이 위까지가 필요한 것 -----  


# sorting

파일 번호 순서대로 픽셀 어레이를 쌓으면 안된다. 나는 처음에 번호 순서대로 쌓았다가 위치순서가 엉망진창임을 몸소 확인했다ㅠㅠ dcm파일을 읽은 후 InstanceNumber를 뽑아 sort한 후 쌓도록 하자.

https://jayeon8282.tistory.com/2

In [ ]:
from glob import glob
import os
import numpy as np
import pydicom
import skimage.external.tifffile import imsave, imread, imshow

dicom_dir = '경 로'

dcm_files = glob(os.path.join(dicom_dir, '*.dcm'))
dcm_files = [pydicom.dcmread(s) for s in dcm_files]

dcm_files.sort(key = lambda x : int(x.InstanceNumber))

dsRef = dcm_files[0]

dcm 파일을 읽으면, 그 안에 많은 key들이 있다. dimension(key : Rows, Columns) 정보 뿐만 아니라, ct vendor사, pixel간 거리(key : PixelSpacing, SliceThickness), pixel array 등이 있으니 필요한 정보가 있으면 key를 뽑아본 후 정보를 확인해 보는 것도 좋은 방법이다.

In [ ]:
"""
see all keys in dicom files
"""

print(dsRef)
print(dsRef.ImageType)

for key in dsRef.__dir__():
	print(key)
    print(key, dsRef.data_element(key))
   

"""
get dimension and thickness information from the dicom file
"""

dims = (len(dcm_files), int(dsRef.Rows), int(dsRef.Columns))
print('dims(z,x,y) : ', dims)

spacing = (float(dsRef.SliceThickness), float(dsRef.PixelSpacing[0]), float(dsRef.PixelSpacing[1]))
print('thickness(z,x,y) : ', spacing)

이제 dimension정보를 얻었으니 sort한 dicom_files를 토대로 pixel_array값을 불러오도록 하자. 

In [ ]:
recon_ct = np.zeros(dims, dtype = dsRef.pixel_array.dtype)

for i, df in enumerate(dicom_files):
	try : 
    	recon_ct[i,:,:] = df.pixel_array
    except : 
    	print(str(i+1).zfill(5) + '.dcm', '**pixel_array_shape Error')

CT영상을 처리하기 위해서는 Hounsfield Units(HU)이 무엇인지 알아야 한다. 이는 X선이 몸을 투과할 때 부위별 흡수정도를 표시한 지표로 CT number라고 부르기도 한다. 물을 0으로 고정하였을 때의 상대적인 흡수량이라고 생각하면 된다(물의 attenuation coefficient에 대한 상대적 비율 * 1000). 우리는 dicom 파일로부터 뽑은 픽셀 어레이를 우선 HU 단위로 정규화 해야한다. 이 때 필요한 것은 dcm파일의 Rescale Slope과 Rescale Intercept 이다. 

 

CT이미지의 경우, 디스크에 저장되는 값과 메모리에 올라오는 값의 표현이 다르게 설정되어 있다. HU는 음수를 포함한 정수값이지만, CT이미지는 일반적으로  unsigned integer인 부호없는 정수로 저장되기 때문이다. 아래의 식은 메모리(output)와 디스크(stored value)에 저장되어 있는 픽셀값의 linear transformation 관계식이다. 

 

(output) = (rescale slope) * (stored value) + (rescale intercept) 

 

우리는 dicom 파일로부터 stored value, rescale slope, rescale intercept을 얻어 위의 식을 계산함으로써 HU에 맞는 값을 얻을 수 있다. (참고 : https://blog.kitware.com/dicom-rescale-intercept-rescale-slope-and-itk/)

In [ ]:
"""
Hounsfield Units(HU)
"""

recon_ct = recon_ct.astype(np.int16)
recon_ct[recon_ct == -2000] = 0

intercept = dsRef.RescaleIntercept
slope = dsRef.RescaleSlope

if slope != 1:
	recon_ct = slope * recon_ct.astype(np.float64)
    recon_ct = recon_ct.astype(np.int16)
    
recon_ct += np.int16(intercept)

HU에 맞는 값을 얻은 후, 무엇을 보고싶은지에 따라 알맞게 window width와 window level을 조정한다. 예를 들어, -1000~+400까지만 보고싶다면, upper bound = 400, lower bound = -1000으로 설정한 후, 0-1로 normalize하여 사용한다.  이렇게 새로 정의된 recon_ct 를 tiff file로 원하는 path에 맞춰 저장하면 된다.

 

In [ ]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0

recon_ct = (recon_ct - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
recon_ct[recon_ct>1] = 1.0
recon_ct[recon_ct<0] = 0.0

recon_ct = np.array(recon_ct, dtype = np.float32)

#save array2tiff
target_path = 'D:/data/tiff'
target_name = 'example.tiff'
target_path = os.path.join(target_path, target_name)

imsave(target_path, recon_ct)

# ----- 위까지 sorting -----
 

In [31]:
sitk.WriteImage(array, 'd.nii.gz')

TypeError: ignored

In [ ]:
# SegBN = np.zeros_like(sitk.GetArrayFromImage(img))

In [ ]:
# SegITK = sitk.GetImageFromArray(SegBN)
# SegITK.CopyInformation(alignedNrd)
# ...
# sitk.WriteImage(SegITK, nrd2niiPath)

In [ ]:
slic_num= len(array)
for slic in range(slic_num):
        DicomFileName = 'pla.dcm'
        sitk.WriteImage(array[slic,:,:], DicomFileName)
        print('Successifully write: ', slic)

TypeError: ignored

In [ ]:
sitk.WriteImage(array, 'plz.dcm')

TypeError: ignored

In [ ]:
array.shape

(121, 512, 512)

In [ ]:
def _transform(dcm_dir,save_dir):
###   from dcm to nii
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dcm_dir)
    reader.SetFileNames(dicom_names)
    image2 = reader.Execute()
    
###   transform 3D image to array
    image_array = sitk.GetArrayFromImage(image2)  #  z,y,x

###   crop the dark voxel
#    new_array,range_list = get_bound(image_array)


###   transform array to 3D image
    image3 = sitk.GetImageFromArray(image_array)

###   save 3D image
    name = dcm_dir.split('/')[-1] + '.nii'
    save_path = os.path.join(save_dir,name)   #   get the save path
    sitk.WriteImage(image3,save_path) 